<a href="https://colab.research.google.com/github/pitaconsumer/Health-pita/blob/master/CAPSTONE_COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COVID-19 Recovery Influenced by East Mediterranean Nutritional Health**


import numpy as np
import pandas as pd
import scipy 
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sqlalchemy import create_engine
import warnings
from pandas_profiling import ProfileReport

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

```
# This is formatted as code
```



In [44]:

!pip install pandas-profiling -U

Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.8.0)


In [0]:
import numpy as np
import pandas as pd
import scipy 
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sqlalchemy import create_engine
import warnings
from pandas_profiling import ProfileReport

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")



Although 86% of countries invest local or donor funds in health security, few countries pay for health security gap assessments and action plans out of national budgets. As such, we hypothesize that recovery rates are not only greatly influenced by the six dimensions of health security described in our analysis below, but also by the nutritional access and regional diets.  For example, the Eastern Mediterranean region consumes high level of nigella seeds, pomegrante seeds as well as almonds and pine nuts (categorized as 'tree nuts' in our Fat Supply Quantity data set), which are regionally believed to boost immunity. Perhaps, this boost in immunity could serve as another layer in fighting COVID-19 and result in higher recovery rates.  

# Upload Global Health Data Sets
## Need to upload the "Fat Supply Quantity" from Kaggle to add to 'Life Expectancy' data set.  
Source: https://www.kaggle.com/mariaren/covid19-healthy-diet-dataset/data https://storage.googleapis.com/kagglesdsdata/datasets/618335/1175348/Fat_Supply_Quantity_Data.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1590357735&Signature=bWQ%2Fj3yRSui127xJMXfJ%2BtW6L0L5xx68ngVOg8U3ewhDulVfTGGUSSVyTSQmO5%2FPnt0%2FAXI7zhGfc6Keejv7bDqaK5mB6pZX4YzqHWPfiBo4s8Oa20eVOXlr2fMFdAZSiOLa6d2fpcia9QfebPZflAH4SgGEO5nuqbJLj%2FDOF0Mv1VHkRRp2BZ8dWsogc2jqxDIqISkOnFxasYodcbzL0lKKzPcfF7QJy%2ByzZj3osBjQ0OXCLwjiAT2DdFdwe%2FQ7Bkthv8fNAW8w9uXExH79t%2BL4QV%2F%2F10BKldwlht3mvzg18ZQYIvrmMtA%2BVLsNYBcvSIH5p%2FJCGN1v6fKnTNZXsw%3D%3D&response-content-disposition=attachment%3B+filename%3DFat_Supply_Quantity_Data.csv

In [49]:
from google.colab import files 
uploaded = files.upload()

Saving Fat_Supply_Quantity_Data.csv to Fat_Supply_Quantity_Data (1).csv


In [0]:
import io 
df = pd.read_csv(io.BytesIO(uploaded['Fat_Supply_Quantity_Data.csv']))

In [0]:
df.head()


In [0]:
df.columns

In [0]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'lifeexpectancy'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

life_df = pd.read_sql_query('select * from lifeexpectancy',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()


In [0]:
life_df

In [0]:
life_df.columns

In [0]:
#df.loc[df['shield'] > 6]
life_df.loc[life_df['Life expectancy '] > 70]

In [0]:
#ProfileReport(df)

In [0]:
#ProfileReport(life_df)

In [0]:
life_df2 = life_df['Year'].max()
life_df2

In [0]:
life_df.nunique()

In [0]:
df.nunique()

In [0]:
#Unique values
df['Recovered'].value_counts()

Exploratory Analysis 
Missing Data


In [52]:
#Drop less used from first data frame: Alcoholic Beverages, Sugar Crops, Sugar & Sweeteners ,Undernourished, Miscellaneous,Aquatic Products, Other,Undernourished,  
df2 = df.drop(['Alcoholic Beverages','Aquatic Products, Other','Sugar & Sweeteners','Sugar Crops','Miscellaneous','Undernourished','Aquatic Products, Other',], axis=1)
df2

,Country,Animal Products,Animal fats,Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,Milk - Excluding Butter,Offals,Oilcrops,Pulses,Spices,Starchy Roots,Stimulants,Treenuts,Vegetal Products,Vegetable Oils,Vegetables,Obesity,Confirmed,Deaths,Recovered,Active,Population,Unit (all except Population)
0,Afghanistan,21.6397,6.2224,8.0353,0.6859,0.0327,0.4246,6.1244,8.2803,0.3103,1.0452,0.1960,0.2776,0.0490,0.0980,0.7513,28.3684,17.0831,0.3593,4.5,0.021411,0.000492,0.002445,0.018474,38042000.0,%
1,Albania,32.0002,3.4172,2.6734,1.6448,0.1445,0.6418,8.7428,17.7576,0.2933,3.1622,0.1148,0.0000,0.0510,0.5270,0.9181,17.9998,9.2443,0.6503,22.3,0.033730,0.001085,0.026522,0.006123,2858000.0,%
2,Algeria,14.4175,0.8972,4.2035,1.2171,0.2008,0.5772,3.8961,8.0934,0.1067,1.1983,0.2698,0.1568,0.1129,0.2886,0.8595,35.5857,27.3606,0.5145,26.6,0.017375,0.001309,0.009142,0.006925,43406000.0,%
3,Angola,15.3041,1.3130,6.5545,0.1539,1.4155,0.3488,11.0268,1.2309,0.1539,3.9902,0.3282,0.0103,0.7078,0.1128,0.0308,34.7010,22.4638,0.1231,6.8,0.000165,0.000010,0.000054,0.000102,31427000.0,%
4,Antigua and Barbuda,27.7033,4.6686,3.2153,0.3872,1.5263,1.2177,14.3202,6.6607,0.1347,1.3579,0.0673,0.3591,0.0449,1.0549,0.2020,22.2995,14.4436,0.2469,19.1,0.025773,0.003093,0.019588,0.003093,97000.0,%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Venezuela (Bolivarian Republic of),16.3261,2.2673,2.5449,0.6555,0.5707,0.9640,7.0949,5.5217,0.2082,0.1003,0.1157,0.0000,0.1311,0.0848,0.0154,33.6855,29.5211,0.1851,25.2,0.002890,0.000035,0.000919,0.001936,28516000.0,%
166,Vietnam,33.2484,3.8238,3.7155,0.7839,1.1217,0.4079,26.4292,0.7520,0.3378,4.2891,0.0765,0.6309,0.0701,0.0829,1.1726,16.7548,5.6211,0.6373,2.1,0.000339,0.000000,0.000275,0.000064,95656000.0,%
167,Yemen,12.5401,2.0131,11.5271,0.5514,0.3847,0.2564,8.0010,1.3463,0.2436,0.7565,0.5001,0.1026,0.0513,0.3718,0.0769,37.4535,23.6312,0.1667,14.1,0.000631,0.000103,0.000017,0.000511,29162000.0,%
168,Zambia,9.6005,1.6113,14.3225,0.6266,1.0070,0.1343,4.9010,1.2756,0.1790,9.8915,0.0783,0.1567,0.2014,0.0671,0.0112,40.3939,15.2848,0.1567,6.5,0.004658,0.000039,0.001103,0.003516,17861000.0,%


In [0]:
df2['Country']

We need to drop columns from dataframes that we have not imputed or filled in with values that produce "Nan". Missing data does not support Decision Tree models. 

In [53]:
#Drop less used from second data frame, Life data frame:
life_df2 = life_df.drop(['Measles ','Hepatitis B','infant deaths',' thinness  1-19 years','Alcohol', ' thinness 5-9 years','GDP','Total expenditure'], axis=1)
life_df2

,Country,Year,Status,Life expectancy,Adult Mortality,percentage expenditure,BMI,under-five deaths,Polio,Diphtheria,HIV/AIDS,Population,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,71.279624,19.1,83,6.0,65.0,0.1,33736494.0,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,73.523582,18.6,86,58.0,62.0,0.1,327582.0,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,73.219243,18.1,89,62.0,64.0,0.1,31731688.0,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,78.184215,17.6,93,67.0,67.0,0.1,3696958.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,7.097109,17.2,97,68.0,68.0,0.1,2978599.0,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,Zimbabwe,2004,Developing,44.3,723.0,0.000000,27.1,42,67.0,65.0,33.6,12777511.0,0.407,9.2
2934,Zimbabwe,2003,Developing,44.5,715.0,0.000000,26.7,41,7.0,68.0,36.7,12633897.0,0.418,9.5
2935,Zimbabwe,2002,Developing,44.8,73.0,0.000000,26.3,40,73.0,71.0,39.8,125525.0,0.427,10.0
2936,Zimbabwe,2001,Developing,45.3,686.0,0.000000,25.9,39,76.0,75.0,42.1,12366165.0,0.427,9.8


**Join datasets by key 'Country' **
We will join both dataframes and select only Year == 2015 from the life_df2 dataframe to create a new dataframe: covid_df.

In [0]:
covid_df = pd.merge(df2, life_df2[life_df2['Year'] == 2015], left_on = 'Country', right_on = 'Country')

We will run our newly merged dataset 'covid_df' that lists country records from 2015, our most recent year. We have 157 individual country records for the models.

In [0]:
covid_df

In [0]:
covid_df.corr()

*Analysis Note*:
We see that Highly correlated =or Above +/- .5 correlation includes 'Schooling' with 'animal fat'.

**Indicator of Health Index:** 
The availability of ventilators and access to critical care beds represents a huge factor towards a country's recovery rate in the COVID-19 pandemic. We found the Global Health Security Index, which is a composite of several healthcare dimensions that include: (1) Prevention; (2) Detection and Reporting; (3) Rapid Response; (4) Health System; (5) Compliance with International Norms; and (6) Risk Environment. A fuller explanation of these dimensions is seen in the note at the bottom. 

To measure the state of state and private health access as well as infrasturcture, we will use the 'Health Index' control for availability of ventilators, or other country financed health interventions and infrastructure to deal with health emergencies, like pandemics.


*Note: The study PREVENTION: Fewer than 7% of countries score in the
highest tier8 for the ability to prevent the emergence or
release of pathogens.
DETECTION AND REPORTING: Only 19% of countries
receive top marks for detection and reporting.
RAPID RESPONSE: Fewer than 5% of countries scored
in the highest tier for their ability to rapidly respond to
and mitigate the spread of an epidemic.
HEALTH SYSTEM: The average score for health system
indicators is 26.4 of 100, making it the lowest-scoring
category.
COMPLIANCE WITH INTERNATIONAL NORMS:
Less than half of countries have submitted
Confidence-Building Measures under the Biological
Weapons Convention (BWC) in the past three years,
an indication of their ability to adhere to important
international norms and commitments related to
biological threats.
RISK ENVIRONMENT: Only 23% of countries score
in the top tier for indicators related to their political
system and government effectiveness.  


~Source: Index: Global Healthy Security Index https://www.ghsindex.org/wp-content/uploads/2019/10/2019-Global-Health-Security-Index.pdf
    
~Variable: Ventilators

~Variable: Critical Care Beds
Saudi Arabia -22.8 per 100,000 (high)
Pakistan - 1.5 (lower-middle)
Iran - 4.6 (upper-middle medium)
Oman - 14.6 (high)
Yemen
Source: https://www.researchgate.net/figure/Number-of-critical-care-beds-per-100-000-population_fig1_338520008
expenditure health - https://link.springer.com/article/10.1007/s00134-012-2627-8

We will define a dictionary that includes the data from the Global Health Security Index report for the 195 countries reporting. Their respective scores will be represented and included as the column 'Health index' in the newly created data set: covid-df.

In [0]:

# Define a dictionary containing Health Index 
health_score_data = {'Country': ['United States','United Kingdom', 'Netherlands', 'Australia','Canada','Thailand','Sweden',
                                 'Denmark','South Korea','Finland','France','Slovenia','Switzerland','Germany','Spain','Norway',
                                 'Latvia','Malaysia','Belgium','Portugal','Japan', 'Brazil','Ireland','Singapore','Argentina',
                                 'Austria','Chile','Mexico','Estonia','Indonesia','Italy','Poland','Lithuania','South Africa',
                                 'Hungary','New Zealand','Greece','Croatia','Albania','Turkey','Serbia','Czech Republic','Georgia',
                                 'Armenia','Ecuador','Mongolia','Kyrgyz Republic','Saudi Arabia','Peru','Vietnam','China','Slovakia',
                                 'Philippines','Israel','Kenya','United Arab Emirates','India','Iceland','Kuwait','Romania',
                                 'Bulgaria','Costa Rica','Russia','Uganda','Colombia','El Salvador','Luxembourg','Montenegro','Morocco',
                                 'Panama','Liechtenstein','Myanmar','Laos','Lebanon','Nicaragua','Oman','Cyprus','Moldova',
                                 'Bosnia and Herzegovina','Jordan','Uruguay','Qatar','Kazakhstan','Ethiopia','Bhutan','Madagascar',
                                 'Egypt','Bahrain','Cambodia','North Macedonia','Dominican Republic','Sierra Leone','Zimbabwe','Ukraine',
                                 'Senegal','Nigeria','Iran','Malta','Trinidad and Tobago','Suriname','Tanzania','Bolivia',
                                 'Paraguay','Namibia',"Côte d'Ivoire",'Ghana','Pakistan','Belarus','St. Lucia','Cuba','Liberia','Nepal',
                                 'Bangladesh','Mauritius','Cameroon','Uzbekistan','Azerbaijan','Gambia','Rwanda','Sri Lanka','Maldives',
                                 'Tunisia','St. Vincent and The Grenadines','Micronesia','Guatemala','Guinea','Monaco','Brunei','Togo',
                                 'Afghanistan','Tajikistan','Niger','Barbados','Seychelles','Belize','Turkmenistan', 'Guyana','Haiti',
                                 'Botswana','San Marino','Swaziland','Bahamas','Andorra','Lesotho','Burkina Faso','Cabo Verde',
                                 'Antigua and Barbuda','Jamaica','Mali','Benin','Chad','Zambia','Mozambique','Malawi',
                                 'Papua New Guinea','Honduras','Grenada','Mauritania','Central African Republic','Comoros','Congo','Samoa',
                                 'St. Kitts and Nevis','Sudan','Vanuatu','Timor-Leste','Iraq','Fiji','Libya','Angola','Tonga',
                                 'Dominica','Algeria','Brazzaville','Djibouti','Venezuela','Burundi','Eritrea','Palau','South Sudan','Tuvalu',
                                 'Nauru','Solomon Islands','Niue','Cook Islands','Gabon','Guinea-Bissau','Syria','Kiribati',
                                 'Yemen','Marshall Islands','São Tomé and Príncipe','North Korea','Somalia','Equatorial Guinea'],
                     
        'Health_index': [83.5,77.9,75.6,75.5,75.3,73.2,72.1,70.4,70.2,68.7,68.2,67.2,67.0,66.0,65.9,64.6,62.9,62.2,61.0,
                        60.3,59.8,59.7,59.0,58.7,58.6,58.5,58.3,57.6,57.0,56.6,56.2,55.4,55.0,54.8,54.0,54.0,53.8,53.3,52.9,52.4,
                        52.3,52.0,52.0,50.2,50.1,49.5,49.3,49.3,49.2,49.1,48.2,47.9,47.6,47.3,47.1,46.7,46.5,46.3,46.1,45.8,45.6,
                        45.1,44.3,44.3,44.2,44.2,43.8,43.7,43.7,43.7,43.5,43.4,43.1,43.1,43.1,43.1,43.0,42.9,42.8,42.1,
                        41.3,41.2,40.7,40.6,40.3,40.1,39.9,39.4,39.2,39.1,38.3,38.2,38.2,38.0,37.9,37.8,37.7,37.3,36.6,36.5,36.4,
                        35.8,35.7,35.6,35.5,35.5,35.5,35.3,35.3,35.2,35.1,35.1,35.0,34.9,34.4,34.3,34.2,34.2,34.2,33.9,33.8,33.7,
                        33.0,32.8,32.7,32.7,32.7,32.6,32.5,32.3,32.3,32.2,31.9,31.9,31.8,31.8,31.7,31.5,31.1,31.1,31.1,30.6,30.5,
                        30.2,30.1,29.3,29.0,29.0,29.0,28.8,28.8,28.7,28.1,28.0,27.8,27.6,27.5,27.5,27.3,27.2,26.5,26.4,26.2,26.2,
                        26.1,26.0,25.8,25.7,25.7,25.2,25.1,24.0,23.6,23.6,23.2,23.0,22.8,22.4,21.9,21.7,21.6,20.8,20.7,20.5,
                        20.4,20.0,20.0,19.9,19.2,18.5,18.2,17.7,17.5,16.6,16.2]} 
  
# Convert the dictionary into DataFrame 
health_df = pd.DataFrame(health_score_data) 
  
# Using DataFrame.insert() to add the column into covid_df
#covid_df.insert(df2["health_score_data"]) 

#Merge the 2 data sets
merged_df = pd.merge(covid_df, health_df, left_on='Country', right_on='Country')

In [0]:
len(health_score_data['Country'])

In [0]:
merged_df.columns

In [0]:
df2['Recovered'].value_counts() 

**Missing Values**
Need to fill in missing values for Percentage Expenditure, BMI, Polio, Treenuts, Pulses, and Vegetables columns. 

In [0]:
life_df2['percentage expenditure'].fillna(life_df2['percentage expenditure'].mean(), inplace=True)

In [0]:
life_df2[' BMI '].fillna(life_df2[' BMI '].mean(), inplace=True)

In [0]:
life_df2['Polio'].fillna(life_df2['Polio'].mean(), inplace=True)

In [0]:
df2['Treenuts'].fillna(df2['Treenuts'].mean(),inplace=True)

In [0]:
df2['Pulses'].fillna(df2['Pulses'].mean(),inplace=True)

In [0]:
df2['Vegetables'].fillna(df2['Vegetables'].mean(),inplace=True)

In [0]:
df2['Meat'].fillna(df2['Meat'].mean(),inplace=True)

We will check the correlation between adjusted variables in new data set.

In [56]:
merged_df.corr()

,Animal Products,Animal fats,Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,Milk - Excluding Butter,Offals,Oilcrops,Pulses,Spices,Starchy Roots,Stimulants,Treenuts,Vegetal Products,Vegetable Oils,Vegetables,Obesity,Confirmed,Deaths,Recovered,Active,Population_x,Year,Life expectancy,Adult Mortality,percentage expenditure,BMI,under-five deaths,Polio,Diphtheria,HIV/AIDS,Population_y,Income composition of resources,Schooling,Health_index
Animal Products,1.000000,0.710333,-0.478648,0.469318,-0.022442,-0.101656,0.736385,0.641814,0.029119,-0.413549,-0.419936,-0.209950,-0.387591,0.495903,0.188401,-1.000000,-0.669238,0.071712,0.435854,0.385509,0.244084,0.348826,0.249087,0.002143,NaN,0.634044,-0.432882,0.118921,0.437910,-0.199489,0.398435,0.295216,-0.375809,-0.180596,0.701782,0.629505,0.515481
Animal fats,0.710333,1.000000,-0.416339,0.262446,-0.116403,-0.159390,0.251783,0.351140,-0.194524,-0.329150,-0.311695,-0.194603,-0.302344,0.296297,0.164782,-0.710321,-0.389604,-0.100507,0.392369,0.363431,0.340994,0.310547,0.241329,0.017692,NaN,0.539625,-0.369972,-0.008903,0.379134,-0.036428,0.289351,0.248572,-0.317849,-0.084778,0.596489,0.563103,0.506550
Cereals - Excluding Beer,-0.478648,-0.416339,1.000000,-0.308149,-0.054167,0.009134,-0.299150,-0.272598,0.279798,0.114859,0.409971,0.100376,0.202710,-0.270974,-0.253826,0.478633,0.023742,-0.012157,-0.489569,-0.358845,-0.296291,-0.294936,-0.263337,0.005893,NaN,-0.553037,0.403518,-0.028851,-0.444254,0.173847,-0.423549,-0.270659,0.515659,0.060297,-0.632969,-0.571845,-0.444253
Eggs,0.469318,0.262446,-0.308149,1.000000,0.223854,-0.052851,0.263722,0.261878,-0.138007,-0.321104,-0.327025,0.011249,-0.350042,0.289669,0.306574,-0.469364,-0.211264,0.183638,0.291984,0.273573,0.118489,0.169268,0.303952,0.130911,NaN,0.565338,-0.389155,0.077561,0.232045,-0.109087,0.274823,0.222723,-0.389517,0.057395,0.560508,0.451998,0.458177
"Fish, Seafood",-0.022442,-0.116403,-0.054167,0.223854,1.000000,0.018571,-0.006628,-0.259120,-0.099272,0.346591,-0.092894,0.221450,0.172502,0.020412,0.201834,0.022396,-0.240058,-0.018436,-0.147099,0.066726,-0.017931,-0.002495,0.148323,-0.005898,NaN,0.055552,-0.046743,-0.073124,-0.113074,-0.062338,-0.010877,-0.042181,-0.077028,0.035187,0.010264,0.001983,-0.068667
Fruits - Excluding Wine,-0.101656,-0.159390,0.009134,-0.052851,0.018571,1.000000,-0.024509,-0.047790,0.079720,0.032919,0.497729,0.003150,0.447195,-0.081584,-0.098389,0.101486,-0.072968,0.033612,-0.088428,-0.053812,-0.065005,-0.044328,-0.035477,-0.040584,NaN,-0.032489,0.036476,0.006071,-0.065394,-0.037703,0.037670,-0.026199,-0.047501,0.023415,-0.092409,-0.057522,-0.104336
Meat,0.736385,0.251783,-0.299150,0.263722,-0.006628,-0.024509,1.000000,0.163367,0.209309,-0.228055,-0.320564,-0.237087,-0.166073,0.290229,-0.038856,-0.736382,-0.548870,-0.026554,0.290266,0.188795,0.050055,0.204846,0.084228,0.016784,NaN,0.320719,-0.166003,-0.018125,0.273449,-0.263508,0.226785,0.094194,-0.137876,-0.192964,0.405613,0.372530,0.244104
Milk - Excluding Butter,0.641814,0.351140,-0.272598,0.261878,-0.259120,-0.047790,0.163367,1.000000,0.019658,-0.403036,-0.188129,-0.062440,-0.395585,0.455033,0.222835,-0.641805,-0.385380,0.278164,0.266737,0.247044,0.160422,0.232973,0.144775,-0.062844,NaN,0.442948,-0.369649,0.321355,0.310507,-0.062141,0.325657,0.315474,-0.312569,-0.122456,0.450697,0.376794,0.343129
Offals,0.029119,-0.194524,0.279798,-0.138007,-0.099272,0.079720,0.209309,0.019658,1.000000,0.027023,0.071100,-0.109078,0.143363,-0.062976,-0.181652,-0.029135,-0.175089,0.036590,-0.257126,-0.147456,-0.159473,-0.144680,-0.066046,0.103447,NaN,-0.312382,0.306215,0.123583,-0.180217,-0.001941,-0.187155,-0.346686,0.298875,-0.039800,-0.271129,-0.253385,-0.175541
Oilcrops,-0.413549,-0.329150,0.114859,-0.321104,0.346591,0.032919,-0.228055,-0.403036,0.027023,1.000000,0.145602,0.127786,0.304605,-0.274782,-0.209238,0.413568,-0.243485,-0.126193,-0.119119,-0.278321,-0.193428,-0.223579,-0.218805,-0.026496,NaN,-0.333776,0.162789,-0.010976,-0.163590,0.038290

In [57]:
merged_df.groupby(['Recovered','Vegetables','Treenuts']).mean

<bound method GroupBy.mean of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f050ce03da0>>

Decision Tree Regression Model

In [58]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score 

#X = merged_df[['percentage expenditure',' BMI ','Polio','Treenuts','Pulses','Meat','Vegetables','Health_index']]
#X = merged_df['Treenuts','Health_index'] #Dropping a bunch to check
merged_df_dropped = merged_df.dropna()
# X = merged_df_dropped[['Treenuts', 'Health_index']]
X = merged_df_dropped.drop(columns=['Recovered', 'Country', 'Unit (all except Population)', 'Status', 'Confirmed', 'Population_x', 'Population_y'])
y = merged_df_dropped['Recovered']

#Split data to train and test with 20% sample 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2,random_state =5)

#regressor = DecisionTreeRegressor()
dtr_model = DecisionTreeRegressor(random_state=0)
dtr_model.fit(X_train, y_train)  

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [0]:
(merged_df_dropped['Recovered'] > 0.5)

Run our score on training set of Decision Tree Regression model.

In [59]:
print(dtr_model.score(X_train, y_train))

0.9999981172875818


Run our score on on test set of Decision Tree Regression model.

In [60]:
print(dtr_model.score(X_test, y_test))

0.10989889981167167


We will identify the most important features in the training set in our Decision Tree Regression model. We see that our dependent variable of interest "Tree Nuts" did not rank in the top 10. This means it does not have as much significance as we hypothesized.

In [61]:
pd.Series(dtr_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

Life expectancy                    0.403328
Pulses                             0.340356
Cereals - Excluding Beer           0.101938
Deaths                             0.080202
Fruits - Excluding Wine            0.017273
Animal fats                        0.013204
Eggs                               0.009714
Stimulants                         0.008631
Milk - Excluding Butter            0.007371
Vegetables                         0.007050
Fish, Seafood                      0.003941
Schooling                          0.001822
Meat                               0.000952
Income composition of resources    0.000938
Obesity                            0.000899
under-five deaths                  0.000726
Polio                              0.000454
Active                             0.000344
Starchy Roots                      0.000193
 BMI                               0.000182
Offals                             0.000128
Vegetable Oils                     0.000112
Adult Mortality                 

Run Second Model: Linear Regression Model
We will use the same target variable 'Recovered' and the above dependent variables. 

In [62]:
from sklearn.linear_model import LinearRegression

line = LinearRegression()

line.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

We will check the score of our Linear Regression model on our same training data set (X_train). 

In [63]:
line.score(X_train, y_train)

0.5045183791666261

We will check the score of our Linear Regression model on our test training set. We see that the score is much lower on our test set. This means that our second model overfit the data to the training set.

In [64]:
line.score(X_test, y_test)

0.14942329478116556

We will set a thresshold of 'Recovery' to 10 percent to see countries that fall into that classifying group.

In [65]:
merged_df[merged_df['Recovered'] <= 0.1]

,Country,Animal Products,Animal fats,Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,Milk - Excluding Butter,Offals,Oilcrops,Pulses,Spices,Starchy Roots,Stimulants,Treenuts,Vegetal Products,Vegetable Oils,Vegetables,Obesity,Confirmed,Deaths,Recovered,Active,Population_x,Unit (all except Population),Year,Status,Life expectancy,Adult Mortality,percentage expenditure,BMI,under-five deaths,Polio,Diphtheria,HIV/AIDS,Population_y,Income composition of resources,Schooling,Health_index
0,Afghanistan,21.6397,6.2224,8.0353,0.6859,0.0327,0.4246,6.1244,8.2803,0.3103,1.0452,0.1960,0.2776,0.0490,0.0980,0.7513,28.3684,17.0831,0.3593,4.5,0.021411,0.000492,0.002445,0.018474,38042000.0,%,2015,Developing,65.0,263.0,71.279624,19.1,83,6.0,65.0,0.1,33736494.0,0.479,10.1,32.3
1,Albania,32.0002,3.4172,2.6734,1.6448,0.1445,0.6418,8.7428,17.7576,0.2933,3.1622,0.1148,0.0000,0.0510,0.5270,0.9181,17.9998,9.2443,0.6503,22.3,0.033730,0.001085,0.026522,0.006123,2858000.0,%,2015,Developing,77.8,74.0,364.975229,58.0,0,99.0,99.0,0.1,28873.0,0.762,14.2,52.9
2,Algeria,14.4175,0.8972,4.2035,1.2171,0.2008,0.5772,3.8961,8.0934,0.1067,1.1983,0.2698,0.1568,0.1129,0.2886,0.8595,35.5857,27.3606,0.5145,26.6,0.017375,0.001309,0.009142,0.006925,43406000.0,%,2015,Developing,75.6,19.0,0.000000,59.5,24,95.0,95.0,0.1,39871528.0,0.743,14.4,23.6
3,Angola,15.3041,1.3130,6.5545,0.1539,1.4155,0.3488,11.0268,1.2309,0.1539,3.9902,0.3282,0.0103,0.7078,0.1128,0.0308,34.7010,22.4638,0.1231,6.8,0.000165,0.000010,0.000054,0.000102,31427000.0,%,2015,Developing,52.4,335.0,0.000000,23.3,98,7.0,64.0,1.9,2785935.0,0.531,11.4,25.2
4,Antigua and Barbuda,27.7033,4.6686,3.2153,0.3872,1.5263,1.2177,14.3202,6.6607,0.1347,1.3579,0.0673,0.3591,0.0449,1.0549,0.2020,22.2995,14.4436,0.2469,19.1,0.025773,0.003093,0.019588,0.003093,97000.0,%,2015,Developing,76.4,13.0,0.000000,47.7,0,86.0,99.0,0.2,NaN,0.784,13.9,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Uruguay,25.5069,3.4811,2.5698,1.2804,0.3281,0.1777,12.2841,8.0603,0.0729,1.5127,0.0729,0.0319,0.1002,0.7564,0.1367,24.4908,18.8819,0.2096,28.9,0.021199,0.000568,0.016709,0.003922,3519000.0,%,2015,Developing,77.0,116.0,0.000000,64.0,0,95.0,95.0,0.1,3431552.0,0.794,15.5,41.3
140,Uzbekistan,25.9903,2.4884,2.7168,1.0639,0.0962,0.5830,10.3624,11.8050,0.1743,0.3366,0.0000,0.0180,0.0902,0.1683,0.7754,24.0067,18.4589,0.8535,15.3,0.008781,0.000039,0.007087,0.001655,33471000.0,%,2015,Developing,69.4,184.0,0.000000,44.7,17,99.0,99.0,0.1,312989.0,0.697,12.1,34.3
142,Yemen,12.5401,2.0131,11.5271,0.5514,0.3847,0.2564,8.0010,1.3463,0.2436,0.7565,0.5001,0.1026,0.0513,0.3718,0.0769,37.4535,23.6312,0.1667,14.1,0.000631,0.000103,0.000017,0.000511,29162000.0,%,2015,Developing,65.7,224.0,0.000000,41.3,47,63.0,69.0,0.1,NaN,0.499,9.0,18.5
143,Zambia,9.6005,1.6113,14.3225,0.6266,1.0070,0.1343,4.9010,1.2756,0.1790,9.8915,0.0783,0.1567,0.2014,0.0671,0.0112,40.3939,15.2848,0.1567,6.5,0.004658,0.000039,0.001103,0.003516,17861000.0,%,2015,Developing,61.8,33.0,0.000000,23.4,40,9.0,9.0,4.1,161587.0,0.576,12.5,28.7


**Sorting Eastern Mediterranean Countries**
We are curious about a region with a high 'Treenuts' content in their daily diet. So we selected the Eastern Mediterranean region from the World Health Organization grouping.

In [0]:
merged_df.iloc[[0,40,70,71,77,83,87,103,116,117,133],'Treenuts'].mean()

In [79]:
merged_df[merged_df['Country'].isin(['Lebanon', 'Afghanistan','Kuwait', 'Pakistan', 'Saudi Arabia','Jordan','Syria','Yemen','Egypt','United Arab Emirates','Oman','Bahrain','Qatar','Morocco','Libya','Tunisia','Somalia','Iran','Iraq'])]

,Country,Animal Products,Animal fats,Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,Milk - Excluding Butter,Offals,Oilcrops,Pulses,Spices,Starchy Roots,Stimulants,Treenuts,Vegetal Products,Vegetable Oils,Vegetables,Obesity,Confirmed,Deaths,Recovered,Active,Population_x,Unit (all except Population),Year,Status,Life expectancy,Adult Mortality,percentage expenditure,BMI,under-five deaths,Polio,Diphtheria,HIV/AIDS,Population_y,Income composition of resources,Schooling,Health_index
0,Afghanistan,21.6397,6.2224,8.0353,0.6859,0.0327,0.4246,6.1244,8.2803,0.3103,1.0452,0.1960,0.2776,0.0490,0.0980,0.7513,28.3684,17.0831,0.3593,4.5,0.021411,0.000492,0.002445,0.018474,38042000.0,%,2015,Developing,65.0,263.0,71.279624,19.1,83,6.0,65.0,0.1,33736494.0,0.479,10.1,32.3
40,Egypt,15.9979,3.4231,13.4988,0.8448,1.3200,0.7216,6.4678,3.6343,0.2992,4.4615,0.1584,0.2376,0.1936,0.0792,0.2992,34.0109,13.6132,0.6600,31.1,0.014363,0.000686,0.004032,0.009645,99064000.0,%,2015,Developing,79.0,159.0,0.000000,61.1,60,93.0,93.0,0.1,NaN,0.688,13.1,39.9
64,Iraq,11.1327,3.8118,6.2761,1.9023,0.1441,0.2306,3.5164,1.7366,0.0288,0.9584,0.1369,0.0576,0.0288,0.7278,0.6341,38.8673,29.6152,0.1513,27.4,0.009473,0.000341,0.006202,0.002931,39310000.0,%,2015,Developing,68.9,182.0,0.000000,59.1,38,63.0,58.0,0.1,36115649.0,0.649,10.1,25.8
70,Jordan,12.2622,1.8696,3.7391,0.5224,0.3189,0.1980,5.8122,3.6512,0.0880,2.5074,0.2584,0.1155,0.0550,0.5994,0.9018,37.7378,29.0553,0.2694,33.4,0.006437,0.000086,0.004272,0.002079,10439000.0,%,2015,Developing,74.1,112.0,0.000000,65.6,4,99.0,99.0,0.1,915932.0,0.741,13.1,42.1
74,Kuwait,19.8923,2.1588,3.0847,2.3242,0.3732,0.3496,9.5706,5.2766,0.1890,1.3416,0.1984,0.5385,0.0472,1.5353,1.1810,30.1101,21.2150,0.5055,37.0,0.417590,0.002947,0.116116,0.298526,4207000.0,%,2015,Developing,74.7,81.0,0.000000,71.4,1,99.0,99.0,0.1,NaN,0.799,13.3,46.1
76,Lebanon,12.4024,2.1637,2.2530,0.3290,0.3681,0.5800,5.1361,4.3553,0.0502,2.8385,0.2175,0.0836,0.1171,0.7250,2.9110,37.5976,27.4593,0.3736,31.3,0.014017,0.000379,0.003661,0.009977,6856000.0,%,2015,Developing,74.9,98.0,0.000000,66.1,1,75.0,81.0,0.1,5851479.0,0.763,13.3,43.1
92,Morocco,15.4227,3.5807,7.1539,1.1517,1.2329,0.7014,6.9989,2.2296,0.2289,0.7309,0.1477,0.3987,0.0960,0.1993,1.9048,34.5736,22.9457,0.2953,25.6,0.020044,0.000545,0.011515,0.007983,35587000.0,%,2015,Developing,74.3,95.0,0.000000,58.5,20,99.0,99.0,0.1,3483322.0,0.645,12.1,43.7
103,Oman,23.6146,2.3712,3.7744,1.4422,0.9290,0.7926,9.0171,9.6407,0.2144,1.1174,0.1689,0.7276,0.0390,1.0979,0.6107,26.3821,17.4495,0.5782,22.9,0.128711,0.000639,0.035378,0.092694,4695000.0,%,2015,Developing,76.6,99.0,0.000000,54.6,1,99.0,99.0,0.1,NaN,0.795,13.7,43.1
104,Pakistan,24.8245,8.9701,4.2124,0.6203,0.0818,0.1227,3.4967,11.5125,0.1431,1.0906,0.2726,0.2249,0.0341,0.0204,0.1909,25.1789,18.8603,0.0750,7.8,0.021194,0.000455,0.006049,0.014689,216565000.0,%,2015,Developing,66.4,161.0,0.000000,25.4,433,72.0,72.0,0.1,18938513.0,0.548,8.1,35.5
114,Saudi Arabia,15.3954,2.6475,5.2126,0.9019,0.4122,0.4219,6.9243,4.4125,0.0970,0.6207,0.1794,0.4800,0.0291,1.2510,0.7128,34.6070,25.4182,0.1891,35.0,0.182784,0.000991,0.097837,0.083956,34218000.0,%,2015,Developing,74.5,88.0,0.000000,68.2,8,97.0,98.0,0.1,NaN,0.845,16.1,49.3


In [80]:
merged_df.loc[[0,40,64,70,74,76,92,103,104,114,133,138,142],'Treenuts']

0      0.7513
40     0.2992
64     0.6341
70     0.9018
74     1.1810
76     2.9110
92     1.9048
103    0.6107
104    0.1909
114    0.7128
133    2.3289
138    3.8246
142    0.0769
Name: Treenuts, dtype: float64

We see that the Eastern Mediterranean countries included (Afghanistan, Egypt, Iraq, Jordan, Kuwait, Lebanon, Morocco, Oman, Pakistan, Saudi Arabia, Tunisia, United Arab Emirates, and Yemen) present a range between .076 (Yemen) to 3.82 (UAE) regarding 'Treenuts' composite in diet. This range may be attributed to income per capita. Specifically, the lowest end of spectrum is Yemen, a lower-income country, in contrast to the highest end of spectrum UAE, a high-income country. We would need to do a comparative analysis to see if the same trend occurs with this diet in another region and see if income disproportionally affects the consumption of tree nuts. 